In [32]:
import os
import traceback
import weka.core.jvm as jvm
from weka.core.converters import Loader,Saver
from weka.classifiers import Classifier, SingleClassifierEnhancer, MultipleClassifiersCombiner, FilteredClassifier, \
    PredictionOutput, Kernel, KernelClassifier
from weka.classifiers import Evaluation
from weka.filters import Filter
#import weka.core.jvm as jvm
from weka.core.classes import Random, from_commandline
import weka.plot.classifiers as plot_cls
import weka.plot.graph as plot_graph
import weka.core.types as types
import weka.filters as filters
import weka.core.serialization as serialization
from weka.core.dataset import Instances

from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#from weka.filters import Filter, MultiFilter, StringToWordVector

valence_actual    = list()
energy_actual     = list()
tension_actual    = list()


#Pooja's directories:
PATH_WAV="F:/set1_wav_actual/"
PATH_DATASET="F:/set1/"
#wekaFilePackage ='C:\Users\Cristina\wekafiles\packages'
#The original arff dataset which should be present in PATH_WAV -- Created in previous python script
dataset = "dataset.arff"
#This is the test dataset for linear regression and it should be present in PATH_WAV
test_linear = 'test_new_logistic_dataset.arff'
#This is the normal test dataset and it should be present in PATH_WAV
test_dataset  = "test_new_dataset.arff"


In [33]:
def filtered_data(data):
    cname  = "weka.filters.unsupervised.attribute.Remove"
    remove = filters.Filter(classname= cname, options=["-R", "1,2,3"])
    remove.inputformat(data)
    filtered = remove.filter(data)
    saver = Saver(classname="weka.core.converters.ArffSaver")
    saver.save_file(filtered, PATH_DATASET + 'new_dataset.arff')
    return filtered

In [34]:
def filtered_data_linear(data,attri):
    #valence = 4
    cname  = "weka.filters.unsupervised.attribute.Reorder"
    #pass_column = '"' + '7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,' + str(valence) + '"'
    #print pass_column
    if(attri == 1):
        remove = filters.Filter(classname= cname, options=["-R", "7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,4"])
    if(attri == 2):
        remove = filters.Filter(classname= cname, options=["-R", "7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,5"])
    if(attri == 3):
        remove = filters.Filter(classname= cname, options=["-R", "7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,6"])
    remove.inputformat(data)
    filtered = remove.filter(data)
    saver = Saver(classname="weka.core.converters.ArffSaver")
    saver.save_file(filtered, PATH_DATASET + 'new_logistic_dataset.arff')
    return filtered

In [35]:
def classification_SMO(filtered):
    filtered.class_is_last()
    classifier = Classifier(classname="weka.classifiers.functions.SMO")
    classifier.build_classifier(filtered)
    return classifier

In [36]:
def classification_linear(filtered_linear):
    filtered_linear.class_is_last()
    #classifierLogistic = Classifier(classname="weka.classifiers.functions.Logistic")
    #
    classifierlinear = Classifier(classname="weka.classifiers.functions.LinearRegression", options=["-S", "1", "-C"])
    classifierlinear.build_classifier(filtered_linear)
    #evaluation = Evaluation(bolts_data)
    #evaluation.crossvalidate_model(classifier, bolts_data, 10, Random(42))
    return classifierlinear
    

In [37]:
def evaluation_Summary(classifier,filtered):
    evaluation = Evaluation(filtered)
    evaluation.crossvalidate_model(classifier, filtered, 10, Random(42))
    #classifier.build_classifier(filtered)
    print("**********Summary Logistic****************")
    print(evaluation.summary())
    print("pctCorrect: " + str(evaluation.percent_correct))
    print("incorrect: " + str(evaluation.incorrect))

In [38]:
def prediction_Model(classifier, data, new_dataset,linear_flag):
    predicted = list()
    global valence_actual, energy_actual, tension_actual
    serialization.write_all(PATH_DATASET + "out.model", [classifier, Instances.template_instances(data)])
    objects = serialization.read_all(PATH_DATASET + "out.model")
    
    classifier2 = Classifier(jobject=objects[0])
    
    ArffTestFile = PATH_DATASET +  new_dataset #"test_new_dataset.arff"
    loader = Loader(classname="weka.core.converters.ArffLoader")
    data = loader.load_file(ArffTestFile)
    data.class_is_last()
    print("# - predicted")
    for index, inst in enumerate(data):
        pred = classifier2.classify_instance(inst)
        dist = classifier2.distribution_for_instance(inst)
        if linear_flag == 0:
            #valence_actual.append(inst.get_string_value(inst.class_index))
            #energy_actual.append(inst.get_string_value(inst.class_index))
            #tension_actual.append(inst.get_string_value(inst.class_index))
            print(
                "%d - %s" %
                (index+1,
                 #inst.get_string_value(inst.class_index),
                 inst.class_attribute.value(int(pred)),
                 #yes" if pred != inst.get_value(inst.class_index) else "no"
                 ))
            predicted.append(inst.class_attribute.value(int(pred)))
        else:
            print(
                "%d - %s" %
                (index+1,
                 str(pred))
                 )
            predicted.append(pred)
    #print valence_actual,energy_actual,tension_actual 
    return predicted

In [39]:
def main():
    #Loading data
    ArffFile  = PATH_DATASET + dataset
    print ArffFile
    jvm.start(class_path = wekaFilePackage)    

    loader = Loader(classname="weka.core.converters.ArffLoader")
    data = loader.load_file(ArffFile)
    
    filtered = filtered_data(data)
    classifier = classification_SMO(filtered)
    evaluation_Summary(classifier,filtered) 
    predicted_mood = prediction_Model(classifier, filtered,test_dataset,0)
    
    #atrribute is 1 for valence model
    attribute = 1
    filtered_linear = filtered_data_linear(data,attribute)
    #removed = remove.filer(data)
    
    classifier_linear = classification_linear(filtered_linear)
    evaluation_Summary(classifier_linear,filtered_linear) 
    predicted_valence = prediction_Model(classifier_linear, filtered_linear,test_linear,1)
    
    #attribute is 2 for energy model
    attribute = 2
    filtered_linear = filtered_data_linear(data,attribute)
    #removed = remove.filer(data)
    
    classifier_linear = classification_linear(filtered_linear)
    evaluation_Summary(classifier_linear,filtered_linear)
    predicted_energy = prediction_Model(classifier_linear, filtered_linear,test_linear,1)

    
    #attribute is 3 for tension model
    attribute = 3
    filtered_linear = filtered_data_linear(data,attribute)
    #removed = remove.filer(data)
    
    classifier_linear = classification_linear(filtered_linear)
    evaluation_Summary(classifier_linear,filtered_linear)
    predicted_tension = prediction_Model(classifier_linear, filtered_linear,test_linear,1)
    #prediction_Model(classifier_logistic, filtered_logistic,"test_new_logistic_dataset.arff")
    
    fig = plt.figure()
    ax = Axes3D(fig)

    #x = [6,3,6,9,12,24]
    #y = [3,5,78,12,23,56]
    # put 0s on the y-axis, and put the y axis on the z-axis
    ax.scatter(xs=predicted_valence, ys=predicted_energy, zs=predicted_tension, zdir='z', label='ys=0, zdir=z')
    plt.show()
    
main()

F:/set1/dataset.arff
**********Summary Logistic****************

Correctly Classified Instances         100               51.5464 %
Incorrectly Classified Instances        94               48.4536 %
Kappa statistic                          0.4268
Mean absolute error                      0.2173
Root mean squared error                  0.3221
Relative absolute error                 89.2945 %
Root relative squared error             92.3552 %
Coverage of cases (0.95 level)          97.4227 %
Mean rel. region size (0.95 level)      72.3122 %
Total Number of Instances              194     

pctCorrect: 51.5463917526
incorrect: 94.0
# - predicted
1 - HAPPY
2 - HAPPY
3 - HAPPY
4 - HAPPY
5 - HAPPY
6 - HAPPY
7 - HAPPY
8 - HAPPY
9 - HAPPY
10 - HAPPY
11 - HAPPY
12 - TENDER
13 - HAPPY
14 - HAPPY
15 - TENDER
16 - HAPPY
17 - HAPPY
18 - HAPPY
19 - HAPPY
20 - TENDER
21 - HAPPY
22 - TENDER
23 - TENDER
24 - HAPPY
25 - SAD
26 - TENDER
27 - HAPPY
28 - SAD
29 - SAD
30 - SAD
31 - SAD
32 - SAD
33 - SAD
34 - S